In [11]:

import pandas as pd 
import geopandas as gpd
import csv
from pathlib import Path

from collections import OrderedDict

In [12]:
data_dir = Path.cwd() / "data"
out_dir = Path.cwd() / "out-files"

In [13]:
# Util method to come between state formats 
# returnType must be either 'full' 'two_digit' or 'FIPS'
def get_state_code(input, returnType = 'FIPS'):
    if (returnType != 'full' and returnType != 'two_digit' and returnType != 'FIPS'):
        raise ValueError("returnType must be either 'full' 'two_digit' or 'FIPS'")

    code_hash = {
        'AL': {'full': 'Alabama', 'two_digit': 'AL', 'FIPS': 1},
        'AK': {'full': 'Alaska', 'two_digit': 'AK', 'FIPS': 2},
        'AZ': {'full': 'Arizona', 'two_digit': 'AZ', 'FIPS': 4},
        'AR': {'full': 'Arkansas', 'two_digit': 'AR', 'FIPS': 5},
        'CA': {'full': 'California', 'two_digit': 'CA', 'FIPS': 6},
        'CO': {'full': 'Colorado', 'two_digit': 'CO', 'FIPS': 8},
        'CT': {'full': 'Connecticut', 'two_digit': 'CT', 'FIPS': 9},
        'DE': {'full': 'Delaware', 'two_digit': 'DE', 'FIPS': 10},
        'DC': {'full': 'District of Columbia', 'two_digit': 'DC', 'FIPS': 11},
        'FL': {'full': 'Florida', 'two_digit': 'FL', 'FIPS': 12},
        'GA': {'full': 'Georgia', 'two_digit': 'GA', 'FIPS': 13},
        'HI': {'full': 'Hawaii', 'two_digit': 'HI', 'FIPS': 15},
        'ID': {'full': 'Idaho', 'two_digit': 'ID', 'FIPS': 16},
        'IL': {'full': 'Illinois', 'two_digit': 'IL', 'FIPS': 17},
        'IN': {'full': 'Indiana', 'two_digit': 'IN', 'FIPS': 18},
        'IA': {'full': 'Iowa', 'two_digit': 'IA', 'FIPS': 19},
        'KS': {'full': 'Kansas', 'two_digit': 'KS', 'FIPS': 20},
        'KY': {'full': 'Kentucky', 'two_digit': 'KY', 'FIPS': 21},
        'LA': {'full': 'Louisiana', 'two_digit': 'LA', 'FIPS': 22},
        'ME': {'full': 'Maine', 'two_digit': 'ME', 'FIPS': 23},
        'MD': {'full': 'Maryland', 'two_digit': 'MD', 'FIPS': 24},
        'MA': {'full': 'Massachusetts', 'two_digit': 'MA', 'FIPS': 25},
        'MI': {'full': 'Michigan', 'two_digit': 'MI', 'FIPS': 26},
        'MN': {'full': 'Minnesota', 'two_digit': 'MN', 'FIPS': 27},
        'MS': {'full': 'Mississippi', 'two_digit': 'MS', 'FIPS': 28},
        'MO': {'full': 'Missouri', 'two_digit': 'MO', 'FIPS': 29},
        'MT': {'full': 'Montana', 'two_digit': 'MT', 'FIPS': 30},
        'NE': {'full': 'Nebraska', 'two_digit': 'NE', 'FIPS': 31},
        'NV': {'full': 'Nevada', 'two_digit': 'NV', 'FIPS': 32},
        'NH': {'full': 'New Hampshire', 'two_digit': 'NH', 'FIPS': 33},
        'NJ': {'full': 'New Jersey', 'two_digit': 'NJ', 'FIPS': 34},
        'NM': {'full': 'New Mexico', 'two_digit': 'NM', 'FIPS': 35},
        'NY': {'full': 'New York', 'two_digit': 'NY', 'FIPS': 36},
        'NC': {'full': 'North Carolina', 'two_digit': 'NC', 'FIPS': 37},
        'ND': {'full': 'North Dakota', 'two_digit': 'ND', 'FIPS': 38},
        'OH': {'full': 'Ohio', 'two_digit': 'OH', 'FIPS': 39},
        'OK': {'full': 'Oklahoma', 'two_digit': 'OK', 'FIPS': 40},
        'OR': {'full': 'Oregon', 'two_digit': 'OR', 'FIPS': 41},
        'PA': {'full': 'Pennsylvania', 'two_digit': 'PA', 'FIPS': 42},
        'RI': {'full': 'Rhode Island', 'two_digit': 'RI', 'FIPS': 44},
        'SC': {'full': 'South Carolina', 'two_digit': 'SC', 'FIPS': 45},
        'SD': {'full': 'South Dakota', 'two_digit': 'SD', 'FIPS': 46},
        'TN': {'full': 'Tennessee', 'two_digit': 'TN', 'FIPS': 47},
        'TX': {'full': 'Texas', 'two_digit': 'TX', 'FIPS': 48},
        'UT': {'full': 'Utah', 'two_digit': 'UT', 'FIPS': 49},
        'VT': {'full': 'Vermont', 'two_digit': 'VT', 'FIPS': 50},
        'VA': {'full': 'Virginia', 'two_digit': 'VA', 'FIPS': 51},
        'WA': {'full': 'Washington', 'two_digit': 'WA', 'FIPS': 53},
        'WV': {'full': 'West Virginia', 'two_digit': 'WV', 'FIPS': 54},
        'WI': {'full': 'Wisconsin', 'two_digit': 'WI', 'FIPS': 55},
        'WY': {'full': 'Wyoming', 'two_digit': 'WY', 'FIPS': 56},
        'PR': {'full': 'Puerto Rico', 'two_digit': 'PR', 'FIPS': 72}
    }
    for row in code_hash:
        for value in code_hash[row]:
            if code_hash[row][value] == input:
                return code_hash[row][returnType]
    return None

In [16]:
# converts district code from the money ball csv to a GEOID
# ex. "CT-HD-59" --> '9059'
# ex. "MN-HD-13A" --> '2713A'
def getGEOID(district_str, leading_zero = False):
    state, chamber, dist_num = district_str.split('-')
    GEOID = str(get_state_code(state)) 
    if leading_zero and len(GEOID) < 2: 
        GEOID= '0' + GEOID
    while len(dist_num) <3: 
        dist_num = '0' + dist_num
    GEOID =  GEOID + dist_num
    return GEOID

In [17]:
# extracts the chamber type from the district code in the money ball csv
# ex. "CT-HD-59" --> 'HD'
def getChamber(district_str):
    state, chamber, dist_num = district_str.split('-')
    return chamber

In [6]:
# extracts the district name.  Used for the non-numerical Massachussets 
# districts in place of GEOID matching
def getName(district_str):
    state, chamber, dist_str = district_str.split('-')
    if len(dist_str) < 4 : return ''
    return dist_str

In [21]:
#################################################
#  PROCESS + ADD FIELDS TO MONEYBALL MODEL CSV  #
#################################################
def process_moneyball_data(inFile, outFile):
    df = pd.read_csv(data_dir / inFile)

    lambdafunc = lambda x: pd.Series(
        getChamber(x['district']),
    )
    df ['chamber'] = df.apply(lambdafunc, axis = 1)

    df.to_csv(data_dir / outFile, index=False, float_format='%.16f')

In [22]:
process_moneyball_data('output_7_28_with_nominees.csv', 'processed_data.csv')

In [84]:
# read in moneyball data
df = pd.read_csv(data_dir / 'processed_data.csv')

# segment to upper and lower chamber
upper_df = df[df['chamber'] == 'SD']
lower_df = df[df['chamber'] == 'HD']

# read in shapefiles 
upper_shp = gpd.read_file(data_dir / 'UPPER_cb_2019_us_sldu_500k/cb_2019_us_sldu_500k.shp')
lower_shp = gpd.read_file(data_dir / 'LOWER_cb_2019_us_sldl_500k/cb_2019_us_sldl_500k.shp')

# read in north carolinas updated district geometry
nc_upper_shp = gpd.read_file(data_dir / 'NC_State_Senate_2020/Senate Consensus Nonpartisan Map v3.shp')
nc_lower_shp = gpd.read_file(data_dir / 'NC_State_House_2020/HB 1020 H Red Comm CSBK-25.shp')

# replace nc geometry from census shapefile
for index, row in upper_shp.iterrows():
    if row['STATEFP'] == '37':
        row['geometry'] = nc_upper_shp[nc_upper_shp['DISTRICT'] == row['NAME']]['geometry']

for index, row in lower_shp.iterrows():
    if row['STATEFP'] == '37':
        row['geometry'] = nc_lower_shp[nc_lower_shp['DISTRICT'] == row['NAME']]['geometry']

In [28]:
upper_shp[upper_shp['STATEFP'] == '37']

,STATEFP,SLDUST,AFFGEOID,GEOID,NAME,LSAD,LSY,ALAND,AWATER,geometry
57,37,035,610U600US37035,37035,35,LU,2018,1509309758,16546712,"POLYGON ((-80.83802 35.00503, -80.83398 35.007..."
96,37,050,610U600US37050,37050,50,LU,2018,7900279494,124102255,"POLYGON ((-84.31749 35.02184, -84.31748 35.021..."
97,37,016,610U600US37016,37016,16,LU,2018,296198870,4401494,"POLYGON ((-78.93421 35.78054, -78.93345 35.783..."
119,37,003,610U600US37003,37003,3,LU,2018,8303498788,564004294,"POLYGON ((-78.51122 36.45481, -78.50721 36.467..."
120,37,034,610U600US37034,37034,34,LU,2018,2354280314,67122426,"POLYGON ((-81.10951 35.77659, -81.10704 35.778..."
159,37,002,610U600US37002,37002,2,LU,2018,4015970003,2886059147,"MULTIPOLYGON (((-76.33014 34.84794, -76.32862 ..."
160,37,042,610U600US37042,37042,42,LU,2018,1708623637,47330685,"POLYGON ((-81.53343 35.57009, -81.50608 35.608..."
251,37,017,610U600US37017,37017,17,LU,2018,698454057,19848839,"POLYGON ((-78.98312 35.64428, -78.98326 35.644..."
252,37,031,610U600US37031,37031,31,LU,2018,1406404644,16866368,"POLYGON ((-80.70770 35.85262, -80.70759 35.853..."
253,37,021,610U600US37021,37021,21,LU,2018,1401976463,7998520,"POLYGON ((-79.45792 35.04456, -79.45570 35.046..."


In [36]:
def pandas_lambda_geolocate(row, df, df_columns, default_values):
    vals = []
    geomatch = df[df['geoid'] == row['GEOID']]
    if len(geomatch.index) < 1:
        #print (f"No match found for GEOID: {row['GEOID']}")
        return pd.Series(default_values)
    elif len(geomatch.index) > 1:
        print(f"More than one match found for GEOID: {row['GEOID']}")
    geomatch = geomatch.iloc[0]
    
    for i in range(0, len(df_columns)):
        vals.append(geomatch[df_columns[i]])

    return pd.Series(vals)

In [37]:
def get_lean(row, df):
    geomatch = df[df['geoid'] == row['GEOID']]
    if len(geomatch.index) < 1:
        return 'no data'
    elif len(geomatch.index) > 1:
        print(f"More than one match found for GEOID: {row['GEOID']}")
    geomatch = geomatch.iloc[0]
    confidence = geomatch['confidence']
    favored = geomatch['favored']
    if confidence == 'Toss-Up': return confidence
    return confidence + " " + favored

In [55]:
# helper method parsing districts with characters
def toInt(foo):
    try:
        int(foo)
    except ValueError:
        return foo
    return int(foo)

In [96]:
# returns full state name, 2 letter postal code, and district from GEOID
def get_district(GEOID, chamber):
    GEOID_str = str(GEOID)
    end_state = 2 if len(GEOID_str) == 5 else 1
    two_letter = get_state_code(int(GEOID_str[0:end_state]), 'two_digit')
    district = two_letter + '-' + chamber + '-' + str(toInt(GEOID_str[end_state:]))
    return district

In [97]:
def get_full_state(GEOID):
    GEOID_str = str(GEOID)
    end_state = 2 if len(GEOID_str) == 5 else 1
    full = get_state_code(int(GEOID_str[0:end_state]), 'full')
    return full

In [98]:
def get_postal(GEOID):
    GEOID_str = str(GEOID)
    end_state = 2 if len(GEOID_str) == 5 else 1
    two_letter = get_state_code(int(GEOID_str[0:end_state]), 'two_digit')
    return two_letter

In [99]:
foo = 20002
#get_location_strings('09013', 'HD')
get_district(foo, 'HD')

'KS-HD-2'

In [86]:
# add data columns from model to geojson 
df_columns = ['district', 'rep_nominee', 'dem_nominee', 'incumbent', 'anti_gerrymandering_party', 'redistricting_voter_power']
default_values = ['',       '',           '',           '',          '',                            0]

upper_shp[['DISTRICT', 'NOM_R', 'NOM_D', 'INCUMBENT','ANTI_GERRY_PARTY', 'VOTER_POWER']] = upper_shp.apply(
    lambda row: pandas_lambda_geolocate(row, upper_df, df_columns, default_values), axis = 1)
upper_shp['LEAN'] = upper_shp.apply(lambda row: get_lean(row, upper_df), axis = 1)

lower_shp[['DISTRICT', 'NOM_R', 'NOM_D', 'INCUMBENT','ANTI_GERRY_PARTY', 'VOTER_POWER']] = lower_shp.apply(
    lambda row: pandas_lambda_geolocate(row, lower_df, df_columns, default_values), axis = 1)
lower_shp['LEAN'] = lower_shp.apply(lambda row: get_lean(row, lower_df), axis = 1)


In [93]:
lower_shp[lower_shp['VOTER_POWER'] == 0].head()

,STATEFP,SLDLST,AFFGEOID,GEOID,NAME,LSAD,LSY,ALAND,AWATER,geometry,DISTRICT,NOM_R,NOM_D,INCUMBENT,ANTI_GERRY_PARTY,VOTER_POWER,LEAN
0,26,054,620L600US26054,26054,54,LL,2018,179945911,6025088,"POLYGON ((-83.66206 42.34603, -83.61740 42.347...",MI-HD-54,,,,,0.0,no data
2,19,048,620L600US19048,19048,48,LL,2018,2712097918,7003089,"POLYGON ((-94.18455 42.47243, -94.17997 42.472...",IA-HD-48,,,,,0.0,no data
3,53,031,620L600US53031,53031,31,LL,2018,2400449381,20088902,"POLYGON ((-122.32560 47.23906, -122.32397 47.2...",WA-HD-31,,,,,0.0,no data
4,53,044,620L600US53044,53044,44,LL,2018,241315251,8022823,"POLYGON ((-122.23892 47.87761, -122.23345 47.8...",WA-HD-44,,,,,0.0,no data
5,53,041,620L600US53041,53041,41,LL,2018,130448372,36757649,"POLYGON ((-122.26661 47.58947, -122.26633 47.5...",WA-HD-41,,,,,0.0,no data


In [100]:
lower_shp['DISTRICT'] = lower_shp.apply(lambda row: get_district(row['GEOID'], 'HD') if row['DISTRICT'] == '' else row['DISTRICT'], axis=1)

In [101]:
lower_shp['FULL'] = lower_shp.apply(lambda row: get_full_state(row['GEOID']), axis=1)
lower_shp['POSTAL'] = lower_shp.apply(lambda row: get_postal(row['GEOID']), axis=1)

In [103]:
lower_shp

,STATEFP,SLDLST,AFFGEOID,GEOID,NAME,LSAD,LSY,ALAND,AWATER,geometry,DISTRICT,NOM_R,NOM_D,INCUMBENT,ANTI_GERRY_PARTY,VOTER_POWER,LEAN,FULL,POSTAL
0,26,054,620L600US26054,26054,54,LL,2018,179945911,6025088,"POLYGON ((-83.66206 42.34603, -83.61740 42.347...",MI-HD-54,,,,,0.000000e+00,no data,Michigan,MI
1,35,043,620L600US35043,35043,43,LL,2018,5150205049,9088041,"POLYGON ((-107.27223 36.33237, -107.27150 36.3...",NM-HD-43,David Hampton,Christine Chandler,D,R,4.292625e-05,Safe D,New Mexico,NM
2,19,048,620L600US19048,19048,48,LL,2018,2712097918,7003089,"POLYGON ((-94.18455 42.47243, -94.17997 42.472...",IA-HD-48,,,,,0.000000e+00,no data,Iowa,IA
3,53,031,620L600US53031,53031,31,LL,2018,2400449381,20088902,"POLYGON ((-122.32560 47.23906, -122.32397 47.2...",WA-HD-31,,,,,0.000000e+00,no data,Washington,WA
4,53,044,620L600US53044,53044,44,LL,2018,241315251,8022823,"POLYGON ((-122.23892 47.87761, -122.23345 47.8...",WA-HD-44,,,,,0.000000e+00,no data,Washington,WA
5,53,041,620L600US53041,53041,41,LL,2018,130448372,36757649,"POLYGON ((-122.26661 47.58947, -122.26633 47.5...",WA-HD-41,,,,,0.000000e+00,no data,Washington,WA
6,16,021,620L600US16021,16021,21,LL,2018,64112839,336525,"POLYGON ((-116.39390 43.53334, -116.39388 43.5...",ID-HD-21,,,,,0.000000e+00,no data,Idaho,ID
7,42,069,620L600US42069,42069,69,LL,2018,2699694356,11468299,"POLYGON ((-79.33040 39.99280, -79.32525 39.999...",PA-HD-69,,,,,0.000000e+00,no data,Pennsylvania,PA
8,50,YO1,620L600US50YO1,50YO1,Windsor-Orange-1,L9,2018,219948463,1881676,"POLYGON ((-72.62029 43.81962, -72.61713 43.823...",VT-HD-YO1,,,,,0.000000e+00,no data,Vermont,VT
9,45,119,620L600US45119,45119,HD-119,LL,2018,141299870,15323523,"POLYGON ((-80.15049 32.60879, -80.14565 32.611...",SC-HD-119,False,Leon Stavrinakis,D,D,2.136959e-05,Safe D,South Carolina,SC


In [72]:
# choose columns you want in output
upper_shp = upper_shp[['STATEFP', 'GEOID', 'DISTRICT', 'NOM_R', 'NOM_D', 'INCUMBENT','ANTI_GERRY_PARTY', 'LEAN', 'VOTER_POWER', 'geometry']]	

upper_shp['NOM_R'].replace({'FALSE': ''}, inplace =True)
upper_shp['NOM_D'].replace({'FALSE': ''}, inplace =True)

In [73]:
upper_shp[upper_shp['VOTER_POWER'] != 0]

,STATEFP,GEOID,DISTRICT,NOM_R,NOM_D,INCUMBENT,ANTI_GERRY_PARTY,LEAN,VOTER_POWER,geometry
5,45,45027,SC-SD-27,Penry Gustafson,Vincent Sheheen,D,D,Likely D,0.052766,"POLYGON ((-80.89719 34.49168, -80.89565 34.493..."
7,45,45026,SC-SD-26,TBA,Nikki Setzler,D,D,Safe D,0.019814,"POLYGON ((-81.76577 33.62673, -81.76369 33.628..."
11,13,13043,GA-SD-43,Melanie Williams,Tonya Anderson,D,D,Safe D,0.001368,"POLYGON ((-84.18805 33.65409, -84.18391 33.655..."
12,13,13017,GA-SD-17,Brian Strickland,Kelly Rose,R,D,Likely R,0.244580,"POLYGON ((-84.35419 33.35336, -84.35418 33.377..."
17,27,27028,MN-SD-28,Jeremy Miller,Sarah Kruger,R,R,Safe R,4.197868,"POLYGON ((-92.44957 43.67444, -92.44953 43.682..."
...,...,...,...,...,...,...,...,...,...,...
1947,09,09011,CT-SD-11,,,D,R,Safe D,0.153386,"POLYGON ((-72.96324 41.43941, -72.95792 41.445..."
1948,09,09022,CT-SD-22,,,D,R,Safe D,0.261162,"POLYGON ((-73.27603 41.30066, -73.27617 41.301..."
1949,09,09015,CT-SD-15,,,D,R,Safe D,0.149968,"POLYGON ((-73.14504 41.55782, -73.13088 41.560..."
1952,55,55018,WI-SD-18,Dan Feyen,Aaron Wojciechowski,R,D,Likely R,0.487346,"POLYGON ((-88.88634 44.04529, -88.88604 44.066..."


In [74]:
upper_shp.to_file(out_dir / "upper_state_moneyball.geojson", driver="GeoJSON")

Previous lower geojson size: 46.8mb
Previous upper geojson size: 32.7mb

In [70]:
pd.set_option('display.max_rows', None)

In [54]:
foo = 'y8'
toInt(foo)

'y8'